In [69]:
from sentence_transformers import SentenceTransformer, util
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_samples, silhouette_score
#from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
#from sklearn.preprocessing import normalize, LabelEncoder, StandardScaler, MinMaxScaler
#from sklearn.manifold import TSNE, LocallyLinearEmbedding, SpectralEmbedding
from sklearn.decomposition import KernelPCA, SparsePCA, TruncatedSVD, PCA
from matplotlib import pyplot as plt
#import matplotlib.cm as cm
import seaborn as sns
import numpy as np
import pandas as pd
import re
import itertools
from autocorrect import Speller #autocorrector

Leemos el dataset

In [70]:
review = pd.read_csv('https://raw.githubusercontent.com/LuisSante/Datasets/main/app_reviews.csv')
review

,package_name,review,date,star
0,com.mantz_it.rfanalyzer,Great app! The new version now works on my Bra...,October 12 2016,4
1,com.mantz_it.rfanalyzer,Great It's not fully optimised and has some is...,August 23 2016,4
2,com.mantz_it.rfanalyzer,Works on a Nexus 6p I'm still messing around w...,August 04 2016,5
3,com.mantz_it.rfanalyzer,The bandwidth seemed to be limited to maximum ...,July 25 2016,3
4,com.mantz_it.rfanalyzer,Works well with my Hackrf Hopefully new update...,July 22 2016,5
...,...,...,...,...
288060,com.termux.api,it doesn't do anything after installing this i...,June 24 2016,3
288061,com.termux.api,I like this app . Its is very helpful for use....,June 20 2016,5
288062,com.termux.api,Finally Brings back the Unix command line to A...,May 20 2016,5
288063,com.termux.api,The API feature is great just need loads more...,May 05 2016,5


In [71]:
def clean_corpus(corpus_review):
        
    for i in range(len(corpus_review)):        
        corpus_review[i] = re.sub(r'https?:\/\/.\S+', "", corpus_review[i]) 
        corpus_review[i] = re.sub(r'"', '', corpus_review[i]) 
        corpus_review[i] = re.sub(r'#', '', corpus_review[i]) 
        corpus_review[i] = re.sub(r'^RT[\s]+', '', corpus_review[i]) 
              
        Apos_dict={"'s":" is","n't":" not","'m":" am","'    ll":" will", 
               "'d":" would","'ve":" have","'re":" are"} #reemplazar apostrofes    
          
        for key,value in Apos_dict.items(): 
            if key in corpus_review[i]: 
                corpus_review[i]=corpus_review[i].replace(key,value) #reemplazar

        corpus_review[i] = " ".join([s for s in re.split("([A-Z][a-z]+[^A-Z]*)",corpus_review[i]) if s])
        corpus_review[i]=corpus_review[i].lower() #minusculas

        file=open("slang.txt","r") #jergas del ingles
        slang=file.read() 
          
        slang=slang.split('\n') 
          
        tweet_tokens= corpus_review[i].split() 
        slang_word=[] 
        meaning=[] 
          
        for line in slang: 
            temp=line.split("=") 
            slang_word.append(temp[0]) 
            meaning.append(temp[-1]) 
          
        for i,word in enumerate(tweet_tokens): 
            if word in slang_word: 
                idx=slang_word.index(word) 
                tweet_tokens[i]=meaning[idx] 
                  
        corpus_review[i]=" ".join(tweet_tokens) 
        corpus_review[i] = ''.join(''.join(s)[:2] for _, s in itertools.groupby(corpus_review[i]))   
 
        spell = Speller(lang='en') 
        corpus_review[i]=spell(corpus_review[i]) 
    return corpus_review
        

Se extrae en un corpus todos los reviews o criticas de usuario

In [72]:
def extract_corpus(dataset):
    lista = []  
    for i in range(len(dataset['package_name'].unique())):
        dataset_temp = dataset.loc[dataset['package_name'] == dataset['package_name'].unique()[i]]
        lista.append({'package_name':dataset['package_name'].unique()[i], 'size': len(dataset_temp)})

    lista = sorted(lista, key=lambda x: x['size'], reverse=True)
    dataframe = dataset[dataset['package_name'] == lista[8]['package_name']]
    corpus = list(dataframe['review'])
    
    corpus = clean_corpus(corpus)

    return corpus

In [73]:
def convert_corpus_to_dataFrame(corpus):
    corpus_ds = {
        'Sentences' : corpus
    }

    dataset_new = pd.DataFrame(corpus_ds)
    return dataset_new

Se crea una funcion que nos permita hacer una incrustacion de palabras con un dataset dado

In [74]:
def neural_embeddings(dataset):
    model_embedder = SentenceTransformer('all-MiniLM-L6-v2')
    corpus = extract_corpus(dataset)

    corpus_embeddings = model_embedder.encode(corpus, convert_to_tensor=True, show_progress_bar=True)
    corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

    return corpus_embeddings, corpus

Creamos tambien la funcion para el metodo de la silueta, este recibe dos argumentos: el dataset y el numero de clusters

In [75]:
def silhoutte(dataset, attempts):

    embeddings, corpus = neural_embeddings(dataset)
    scores_silhouette = []

    for k in range(2,attempts):

        agglomerative_clusterering = AgglomerativeClustering(n_clusters=k, affinity="cosine" , linkage="complete").fit(embeddings)
        cluster_labels = agglomerative_clusterering.labels_

        silhouette_avg = silhouette_score(embeddings, cluster_labels)
        scores_silhouette.append(silhouette_avg)

    max_score = max(scores_silhouette)
    max_index = scores_silhouette.index(max_score)
    n_clusters = max_index + 2

    return n_clusters, embeddings, corpus

Se hace una funcion para la segmentación del texto

In [76]:
def segmentation(dataset_review, attempts):
    n_clusters, embeddings, corpus = silhoutte(dataset_review, attempts)

    agglomerative_clusterering = AgglomerativeClustering(n_clusters=n_clusters, affinity="cosine" , linkage="complete").fit(embeddings)
    cluster_labels = agglomerative_clusterering.labels_

    return n_clusters, cluster_labels, corpus

Esta funcion se dedica a realizar el clustering del dataset 

In [77]:

def clustering(dataset_review, attempts):
    n_clusters, labels, corpus = segmentation(dataset_review, attempts)

    corpus_dataframe = convert_corpus_to_dataFrame(corpus)
    corpus_dataframe['clusters'] = labels
    corpus_dataframe = corpus_dataframe.sort_values(by=['clusters'])

    '''clustered_sentences = {}
    for sentence_id, cluster_id in enumerate(labels):
        if cluster_id not in clustered_sentences:
            clustered_sentences[cluster_id] = []
    
        clustered_sentences[cluster_id].append(corpus[sentence_id])
    
    for i, cluster in clustered_sentences.items():
        print("Cluster ", i+1)
        print(cluster)
        print("     ")'''

Por ultimo se muestra el dataset final

In [78]:
def red_dim(corpus_embeddings):
    scaler = PCA(n_components=2, random_state = 100)
    X_principal = scaler.fit_transform(corpus_embeddings)
    X_principal.shape
    X_principal
    distribution = pd.DataFrame(X_principal, columns=['x', 'y'])
    distribution
    
    return X_principal, distribution

In [79]:
def show_dimentions(distribution):
    for col in 'xy':
        sns.kdeplot(distribution[col], shade=True)

    with sns.axes_style(style='ticks'):
       g = sns.factorplot(data=distribution, kind="box")

In [80]:
def show_graphics(corpus_embeddings,X_principal,clustering_model):
    point_size = 100.0 / np.sqrt(corpus_embeddings.shape[0])
    result = pd.DataFrame(X_principal, columns=['x', 'y'])
    result['labels'] = clustering_model.labels_
    print(result)
    
    fig, ax = plt.subplots(figsize=(14, 8))
    clustered = result[result.labels != -1]
    plt.scatter(clustered.x, clustered.y, c=clustered.labels, s=20, cmap='Spectral')
    plt.colorbar()
    plt.show()

In [81]:
clustering(review, 10)

Batches:   0%|          | 0/93 [00:00<?, ?it/s]

Cluster  2
['veryold', 'عالی good', 'the best ever', 'awesome. love it 😜😜😁😁😜😁😁', 'i am feeling very well', 'loving it.. very important and handy..', 'does not work ca not add any accounts.', 'it is easy to use like child is play', "not working exactly.. i'hv just lost access of 3 imp accounts..admin please help to access my accounts..its important", 'i am not able to install. google play on phone says waiting for network and website says its already installed. will change the rating after installing.', 'what is the key when you add an account you put your email address? the the key df is the key i ca not member how it is done', 'ca not download it off mobile data or and google apps with google at the beginning of name with a google phone no less. nothing corrects this issue resting or anything it just stalls in middle of dl', 'cannot open - crashes immediately', 'ca not move it to my sd card too many apps ca not be moved to the sd card most of which are built by google so.. just like s